In [ ]:
pip install contractions

     |████████████████████████████████| 327kB 7.3MB/s 
     |████████████████████████████████| 266kB 9.8MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85394 sha256=9c2c828faae8223b344900b83333c93b66cd7265d577a711bf72197fe83424c2
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [ ]:
import pickle
import nltk
import pandas as pd
nltk.download('punkt')
nltk.download('wordnet')
import re
import numpy as np
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
nltk.download ('stopwords')
from nltk.corpus import stopwords 
import glob
import ntpath
import contractions
from google.colab import drive
from nltk.stem import WordNetLemmatizer 
import operator

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import os
drive.mount('/content/drive')
files_list = glob.glob("drive/My Drive/stories/stories/" + '**/*', recursive=True)   
print(files_list) 

Mounted at /content/drive
['drive/My Drive/stories/stories/17.lws', 'drive/My Drive/stories/stories/16.lws', 'drive/My Drive/stories/stories/100west.txt', 'drive/My Drive/stories/stories/14.lws', 'drive/My Drive/stories/stories/13chil.txt', 'drive/My Drive/stories/stories/6napolen.txt', 'drive/My Drive/stories/stories/19.lws', 'drive/My Drive/stories/stories/6ablemen.txt', 'drive/My Drive/stories/stories/3gables.txt', 'drive/My Drive/stories/stories/3sonnets.vrs', 'drive/My Drive/stories/stories/4moons.txt', 'drive/My Drive/stories/stories/7oldsamr.txt', 'drive/My Drive/stories/stories/3lpigs.txt', 'drive/My Drive/stories/stories/3student.txt', 'drive/My Drive/stories/stories/20.lws', 'drive/My Drive/stories/stories/18.lws', 'drive/My Drive/stories/stories/3wishes.txt', 'drive/My Drive/stories/stories/5orange.txt', 'drive/My Drive/stories/stories/advtthum.txt', 'drive/My Drive/stories/stories/adv_alad.txt', 'drive/My Drive/stories/stories/7voysinb.txt', 'drive/My Drive/stories/stories/

In [ ]:
files_list.remove("drive/My Drive/stories/stories/SRE")
len(files_list)

467

Preprocessing

In [ ]:
#Function for preprocessing
def pre_processing(file_path, temp):
  tokens=[]
  if(temp==0):                                         ## parameter is a file
    rfile = open(file_path,'r',errors = 'ignore')
    read_file = rfile.read().replace('\n', ' ')
    clean_string = re.sub(r'[\!@#$%^&\*()\_+={}\:\;<>\?/\|\,\-/"\']'," ",read_file)
    clean_string = re.sub('\s+', ' ', clean_string)
    clean_string = re.sub('^\s+', '', clean_string)
    clean_string = re.sub("([\-]{1,2})", " ",clean_string)
    clean_string = re.sub('([>|\-=+~]+)', ' ', clean_string)
    clean_string=re.sub("([\.\!]{2})", " ",clean_string)
    clean_string=re.sub("[^a-zA-Z\s\n]", " ",clean_string)  
    read_file = re.sub(r"(.)\1{2}","",clean_string)
  else:                                               ## paramter is a query
    read_file=file_path                     

  #convert to lowercase
  read_file = read_file.lower() 

  #Tokenisation
  sent_tokens = sent_tokenize(read_file)   
  strings = ''  
  
  #Tokenize sentences into words
  for w in sent_tokens:
    word = w.split(' ')
    clean_string = ' '.join(word)          

    #Remove stopwords from tokens
    clean_word = clean_string.split(' ')
    stop_words = set(stopwords.words('english'))
    for x in clean_word:
      if x not in stop_words and len(x)>1:
        tokens.append(x)
  return tokens

Read file and document frequency

In [ ]:
doc=0
pprtext = []
file_words={}
for file_path in files_list:
  doc_id=ntpath.basename(file_path)
  tokens=pre_processing(file_path,0)
  file_words[doc]=doc_id
  pprtext.append(tokens)
  doc=doc+1
  print(doc)

document_frequency={}
for i in range(len(pprtext)):
  for j in set(pprtext[i]):
    if(j not in document_frequency):
      document_frequency[j]=[1,[i]]
    else:
      document_frequency[j][0]+=1
      document_frequency[j][1].append(i)

In [ ]:
document_frequency

Term Frequency

In [ ]:
term_frequency={}
max_freq=[]
for i in range(len(pprtext)):
  max=-1
  term_frequency[i]=[0, {}]
  for j in pprtext[i]:
    term_frequency[i][0]+=1
    if(j not in term_frequency[i][1]):
      term_frequency[i][1][j]=1
    else:
      term_frequency[i][1][j]+=1
    if(max<term_frequency[i][1][j]):
      max=term_frequency[i][1][j]
  max_freq.append(max)

In [ ]:
term_frequency

In [ ]:
max_freq

BOOLEAN MODEL



In [ ]:
boolean={}
for token in document_frequency:
  boolean[token]=[0]*467
  for i in document_frequency[token][1]:
    boolean[token][i]=1

In [ ]:
boolean

RAW COUNT model

In [ ]:
raw_count={}
for token in document_frequency:
  raw_count[token]=[0]*467
  for i in range(467):
    if(token in term_frequency[i][1]):
      raw_count[token][i]=term_frequency[i][1][token]

In [ ]:
raw_count

{}

Term frequency model

In [ ]:
term_freq={}
for token in document_frequency:
  term_freq[token]=[0]*467
  for i in range(467):
    if(token in term_frequency[i][1]):
      term_freq[token][i]=term_frequency[i][1][token]/term_frequency[i][0]

In [ ]:
term_freq

{}

LOG NORMALISATION MODEL


In [ ]:
log_normalisation={}
for token in document_frequency:
  log_normalisation[token]=[0]*467
  for i in range(467):
    if(token in term_frequency[i][1]):
      log_normalisation[token][i]=np.log2(1+term_frequency[i][1][token])

In [ ]:
log_normalisation

{}

DOUBLE NORMALISATION MODEL

In [ ]:
double_normalisation={}
for token in document_frequency:
  double_normalisation[token]=[0]*467
  for i in range(467):
    if(token in term_frequency[i][1]):
      double_normalisation[token][i]=0.5+0.5*(term_frequency[i][1][token]/max_freq[i])

In [ ]:
double_normalisation

{}

IDF_list

In [ ]:
IDF_list={}
for token in document_frequency:
  IDF_list[token]=np.log2(467/(document_frequency[token][0]+1))
print(IDF_list)

{'went': 0.628874000384583, 'threw': 2.0859190261850022, 'heard': 0.9072768076415809, 'late': 1.5274287368250372, 'pile': 2.8228846203512084, 'employment': 4.619351226266076, 'programmer': 4.867278739709662, 'nothing': 0.8899988162097455, 'ground': 1.5097267350915782, 'ratty': 5.697353738267349, 'eternity': 3.6973537382673496, 'decorating': 6.059923817652058, 'destroyed': 2.8899988162097454, 'things': 1.0030925950553815, 'contracts': 5.16683902156857, 'screamed': 2.2234225499349374, 'teach': 3.619351226266076, 'rest': 1.2898499116739133, 'pass': 2.1668390215685696, 'post': 3.00929774458209, 'alley': 3.913082429322787, 'hold': 1.7379957227646954, 'director': 4.343716783652649, 'faith': 3.913082429322787, 'corporate': 5.282316238988505, 'feet': 1.2898499116739133, 'legal': 4.059923817652058, 'computer': 2.424335243860934, 'head': 0.6873696496947275, 'swept': 2.8899988162097454, 'although': 2.1530332220435393, 'politics': 4.407847121072365, 'managerial': 7.867278739709662, 'truce': 5.5453

In [ ]:
len(IDF_list)

0

Saving the data

In [ ]:
out=open("/content/drive/My Drive/IR/IDF_list", 'wb')
pickle.dump(IDF_list,out)
out.close()
out=open("/content/drive/My Drive/IR/Double_normalisation", 'wb')
pickle.dump(double_normalisation,out)
out.close()
out=open("/content/drive/My Drive/IR/log_normalisation", 'wb')
pickle.dump(log_normalisation,out)
out.close()
out=open("/content/drive/My Drive/IR/term_freq", 'wb')
pickle.dump(term_freq,out)
out.close()
out=open("/content/drive/My Drive/IR/raw_count", 'wb')
pickle.dump(raw_count,out)
out.close()
out=open("/content/drive/My Drive/IR/boolean", 'wb')
pickle.dump(boolean,out)
out.close()
out=open("/content/drive/My Drive/IR/term_frequency", 'wb')
pickle.dump(term_frequency,out)
out.close()
out=open("/content/drive/My Drive/IR/document_frequency", 'wb')
pickle.dump(document_frequency,out)
out.close()
out=open("/content/drive/My Drive/IR/file_words", 'wb')
pickle.dump(file_words,out)
out.close()

Loading the data

In [ ]:
out=open("/content/drive/My Drive/IR/IDF_list", 'rb')
IDF_list=pickle.load(out)
out.close()
out=open("/content/drive/My Drive/IR/Double_normalisation", 'rb')
double_normalisation=pickle.load(out)
out.close()
out=open("/content/drive/My Drive/IR/log_normalisation", 'rb')
log_normalisation=pickle.load(out)
out.close()
out=open("/content/drive/My Drive/IR/term_freq", 'rb')
term_freq=pickle.load(out)
out.close()
out=open("/content/drive/My Drive/IR/raw_count", 'rb')
raw_count=pickle.load(out)
out.close()
out=open("/content/drive/My Drive/IR/boolean", 'rb')
boolean=pickle.load(out)
out.close()
out=open("/content/drive/My Drive/IR/term_frequency", 'rb')
term_frequency=pickle.load(out)
out.close()
out=open("/content/drive/My Drive/IR/document_frequency", 'rb')
document_frequency=pickle.load(out)
out.close()
out=open("/content/drive/My Drive/IR/file_words", 'rb')
file_words=pickle.load(out)
out.close()

**Calculation of TF_IDF values for each word for each model**


Boolean model

In [ ]:
tfidf_boolean={}
for i in document_frequency:
  tfidf_boolean[i]=[]
  for j in range(467):
    tfidf_boolean[i].append(IDF_list[i]*boolean[i][j])

In [ ]:
tfidf_boolean

raw_count


In [ ]:
tfidf_rawcount={}
for i in document_frequency:
  tfidf_rawcount[i]=[]
  for j in range(467):
    tfidf_rawcount[i].append(IDF_list[i]*raw_count[i][j])

In [ ]:
tfidf_rawcount

term frequency

In [ ]:
tfidf_tfreq={}
for i in document_frequency:
  tfidf_tfreq[i]=[]
  for j in range(467):
    tfidf_tfreq[i].append(IDF_list[i]*term_freq[i][j])

In [ ]:
tfidf_tfreq

log-normalisation


In [ ]:
tfidf_norm={}
for i in document_frequency:
  tfidf_norm[i]=[]
  for j in range(467):
    tfidf_norm[i].append(IDF_list[i]*log_normalisation[i][j])


In [ ]:
tfidf_norm

Double normalisation

In [ ]:
tfidf_doublenorm={}
for i in document_frequency:
  tfidf_doublenorm[i]=[]
  for j in range(467):
    tfidf_doublenorm[i].append(IDF_list[i]*double_normalisation[i][j])

In [ ]:
tfidf_doublenorm

Query

In [ ]:
query=input("Input the query")
query_token=pre_processing(query, 1)

Input the queryking kong


In [ ]:
query_token

['king', 'kong']

boolean model retrieval

In [ ]:
documents_boolean={}
for i in range(467):
  documents_boolean[i]=0
  for j in query_token:
    if(j in boolean):
      documents_boolean[i]+=tfidf_boolean[j][i]

doc_boolean = list(reversed(sorted(documents_boolean.items(), key = lambda kv:(kv[1], kv[0]))))
print(doc_boolean)

[(6, 9.430776731242222), (406, 6.867278739709662), (284, 6.867278739709662), (450, 2.563497991532559), (448, 2.563497991532559), (445, 2.563497991532559), (439, 2.563497991532559), (419, 2.563497991532559), (418, 2.563497991532559), (413, 2.563497991532559), (403, 2.563497991532559), (401, 2.563497991532559), (383, 2.563497991532559), (382, 2.563497991532559), (381, 2.563497991532559), (379, 2.563497991532559), (369, 2.563497991532559), (358, 2.563497991532559), (339, 2.563497991532559), (335, 2.563497991532559), (313, 2.563497991532559), (309, 2.563497991532559), (291, 2.563497991532559), (288, 2.563497991532559), (287, 2.563497991532559), (283, 2.563497991532559), (282, 2.563497991532559), (278, 2.563497991532559), (275, 2.563497991532559), (274, 2.563497991532559), (271, 2.563497991532559), (270, 2.563497991532559), (269, 2.563497991532559), (268, 2.563497991532559), (248, 2.563497991532559), (240, 2.563497991532559), (237, 2.563497991532559), (234, 2.563497991532559), (229, 2.56349

In [ ]:
for j in range(5):
  print(file_words[doc_boolean[j][0]])

19.lws
superg1
mario.txt
yukon.txt
vday.hum


Raw_count retrieval

In [ ]:
documents_rawcount={}
for i in range(467):
  documents_rawcount[i]=0
  for j in query_token:
    if(j in raw_count):
      documents_rawcount[i]+=tfidf_rawcount[j][i]

doc_rawcount = list(reversed(sorted(documents_rawcount.items(), key = lambda kv:(kv[1], kv[0]))))
print(doc_rawcount)



[(220, 228.15132124639777), (226, 97.41292367823725), (28, 66.65094777984653), (171, 56.3969558137163), (161, 56.3969558137163), (20, 41.015967864520945), (339, 38.45246987298839), (309, 35.88897188145583), (22, 35.88897188145583), (401, 33.325473889923266), (7, 30.76197589839071), (382, 28.19847790685815), (283, 28.19847790685815), (274, 25.63497991532559), (134, 25.63497991532559), (291, 23.07148192379303), (284, 20.601836219128987), (18, 20.507983932260473), (418, 15.380987949195355), (379, 10.253991966130236), (369, 10.253991966130236), (313, 10.253991966130236), (287, 10.253991966130236), (240, 10.253991966130236), (208, 10.253991966130236), (130, 10.253991966130236), (6, 9.430776731242222), (403, 7.690493974597677), (271, 7.690493974597677), (234, 7.690493974597677), (178, 7.690493974597677), (85, 7.690493974597677), (68, 7.690493974597677), (406, 6.867278739709662), (450, 5.126995983065118), (448, 5.126995983065118), (270, 5.126995983065118), (269, 5.126995983065118), (268, 5.12

In [ ]:
for j in range(5):
  print(file_words[doc_rawcount[j][0]])

gulliver.txt
hop-frog.poe
aesop11.txt
fgoose.txt
fable.txt


Term_frequency retrieval

In [ ]:
documents_tf={}
for i in range(467):
  documents_tf[i]=0
  for j in query_token:
    if(j in term_freq):
      documents_tf[i]+=tfidf_tfreq[j][i]

doc_tf = list(reversed(sorted(documents_tf.items(), key = lambda kv:(kv[1], kv[0]))))
print(doc_tf)

[(291, 0.09776051662624166), (339, 0.06915911847659781), (274, 0.06252434125689169), (161, 0.05589391061815292), (226, 0.05528542774020275), (18, 0.05299220654330872), (283, 0.049997301253294595), (7, 0.04890616200062115), (309, 0.04772469665087211), (284, 0.04598624156055577), (418, 0.0433267266174517), (401, 0.04191883508166449), (178, 0.03964172151854473), (270, 0.03487752369432053), (268, 0.030517833232530465), (208, 0.023681274748568674), (287, 0.022437619181904237), (313, 0.014301244025286243), (20, 0.011280519214664726), (179, 0.010770999964422517), (171, 0.008414944167967219), (117, 0.007473755077354399), (81, 0.007023282168582353), (22, 0.0063419282349276955), (382, 0.006305563038206206), (86, 0.006103566646506094), (6, 0.00590161247261716), (219, 0.005812920615720089), (144, 0.005646471346988016), (448, 0.005483418163706009), (275, 0.004948837821491427), (123, 0.004948837821491427), (383, 0.0049109156925911095), (220, 0.00474711972797898), (269, 0.00455732976272455), (419, 0.

In [ ]:
for j in range(5):
  print(file_words[doc_tf[j][0]])

monkking.txt
pussboot.txt
lpeargrl.txt
fable.txt
hop-frog.poe


Normalisation retrieval

In [ ]:
documents_norm={}
for i in range(467):
  documents_norm[i]=0
  for j in query_token:
    if(j in log_normalisation):
      documents_norm[i]+=tfidf_norm[j][i]

doc_norm = list(reversed(sorted(documents_norm.items(), key = lambda kv:(kv[1], kv[0]))))
print(doc_norm)

[(220, 16.641852373765545), (284, 13.734557479419324), (226, 13.549117972495106), (28, 12.189144561759317), (171, 11.596141988925247), (161, 11.596141988925247), (20, 10.478202784009211), (339, 10.253991966130236), (309, 10.015306194979882), (22, 10.015306194979882), (401, 9.760146695746272), (7, 9.486069785241998), (6, 9.430776731242222), (382, 9.190044170318224), (283, 9.190044170318224), (274, 8.868246006220941), (134, 8.868246006220941), (291, 8.515755999259333), (18, 8.126096374506211), (418, 7.196648704213712), (406, 6.867278739709662), (379, 5.952258007726774), (369, 5.952258007726774), (313, 5.952258007726774), (287, 5.952258007726774), (240, 5.952258007726774), (208, 5.952258007726774), (130, 5.952258007726774), (403, 5.126995983065118), (271, 5.126995983065118), (234, 5.126995983065118), (178, 5.126995983065118), (85, 5.126995983065118), (68, 5.126995983065118), (450, 4.063048187253106), (448, 4.063048187253106), (270, 4.063048187253106), (269, 4.063048187253106), (268, 4.063

In [ ]:
for j in range(5):
  print(file_words[doc_norm[j][0]])

gulliver.txt
mario.txt
hop-frog.poe
aesop11.txt
fgoose.txt


Double Normalisation retrieval

In [ ]:
documents_doublenorm={}
for i in range(467):
  documents_doublenorm[i]=0
  for j in query_token:
    if(j in double_normalisation):
      documents_doublenorm[i]+=tfidf_doublenorm[j][i]

doc_doublenorm = list(reversed(sorted(documents_doublenorm.items(), key = lambda kv:(kv[1], kv[0]))))
print(doc_doublenorm)

[(6, 4.858278922155083), (284, 3.8015293023392775), (406, 3.5239983006404847), (291, 2.563497991532559), (226, 2.563497991532559), (161, 2.563497991532559), (7, 2.464901914935153), (401, 2.3925981254303883), (339, 2.197283992742194), (283, 2.1629514303555966), (274, 2.082842118120204), (309, 1.9226234936494193), (18, 1.9226234936494193), (20, 1.8360188317733195), (418, 1.7341309942720253), (208, 1.708998661021706), (171, 1.702621800346252), (270, 1.647962994556645), (178, 1.6021862447078494), (220, 1.571281129327698), (81, 1.5380987949195355), (382, 1.5207191475193147), (313, 1.514794267723785), (287, 1.514794267723785), (268, 1.4789411489610917), (448, 1.4648559951614621), (86, 1.4648559951614621), (28, 1.4374755092705938), (248, 1.4241655508514217), (134, 1.4181052719116285), (22, 1.4136937453304554), (383, 1.4099238953429076), (43, 1.4038203286964015), (89, 1.3733024954638708), (237, 1.361858308001672), (271, 1.3602234240785007), (419, 1.3571459955172371), (269, 1.3571459955172371),

In [ ]:
for j in range(5):
  print(file_words[doc_doublenorm[j][0]])

19.lws
mario.txt
superg1
monkking.txt
hop-frog.poe


# ***COSINE*** ***SIMILARITY***

In [ ]:
query_vector={}
for i in document_frequency:
  query_vector[i]=0

for i in query_token:
  if(i in query_vector):
    query_vector[i]=1

sum=0
for i in query_vector:
  sum+=query_vector[i]**2

##normalisation the query vector
for i in query_vector:
  query_vector[i]=query_vector[i]/(sum**0.5)

In [ ]:
query_vector

BOOLEAN

In [ ]:
cosine_score={}
for i in range(467):
  sum=0
  prod=0
  prod_sum=0
  for j in tfidf_boolean:
    sum+=tfidf_boolean[j][i]**2
    prod=prod+query_vector[j]*tfidf_boolean[j][i]
  print(i)
  cosine_score[i]=(prod/(sum**0.5))
  

In [ ]:
cosine_scores = list(reversed(sorted(cosine_score.items(), key = lambda kv:(kv[1], kv[0]))))
print(cosine_scores)

[(284, 0.05570767028073486), (6, 0.05231086266918095), (270, 0.047948554357919636), (268, 0.04565368033684614), (291, 0.04541845054483657), (178, 0.045324951206304004), (418, 0.041749086018582016), (219, 0.03886573552533822), (117, 0.037547064691757684), (339, 0.03468984697474707), (208, 0.0345152577377579), (274, 0.03392027832907927), (287, 0.0336640619436372), (144, 0.031241467976362687), (383, 0.030623274109937836), (18, 0.029466431422982196), (7, 0.02935847162620516), (81, 0.028917720582596437), (179, 0.02775945355318816), (309, 0.027094266839995714), (86, 0.026072836667027623), (419, 0.02595107086595904), (313, 0.02584543963479218), (275, 0.02427505150661677), (123, 0.02427505150661677), (135, 0.02356256859013792), (406, 0.02336571459932093), (248, 0.022053517622601295), (66, 0.02118557023699066), (283, 0.02111233180355076), (401, 0.021081619750206032), (269, 0.020409014975286993), (335, 0.01927670420436139), (448, 0.01854592208166064), (89, 0.01778607956376981), (229, 0.017618603

In [ ]:
for i in range(5):
  print(file_words[cosine_scores[i][0]])

mario.txt
19.lws
lionwar.txt
lionmosq.txt
monkking.txt


RAW_COUNT

In [ ]:
cosine_score_2={}
for i in range(467):
  sum=0
  prod=0
  prod_sum=0
  for j in tfidf_rawcount:
    sum+=tfidf_rawcount[j][i]**2
    prod=prod+query_vector[j]*tfidf_rawcount[j][i]
  print(i)
  cosine_score_2[i]=(prod/(sum**0.5))

In [ ]:
cosine_scores_2 = list(reversed(sorted(cosine_score_2.items(), key = lambda kv:(kv[1], kv[0]))))
for i in range(5):
  print(file_words[cosine_scores_2[i][0]])

fable.txt
hop-frog.poe
monkking.txt
6ablemen.txt
pussboot.txt


TERM_FREQUENCY

In [ ]:
cosine_score_3={}
for i in range(467):
  sum=0
  prod=0
  prod_sum=0
  for j in tfidf_tfreq:
    sum+=tfidf_tfreq[j][i]**2
    prod=prod+query_vector[j]*tfidf_tfreq[j][i]
  print(i)
  cosine_score_3[i]=(prod/(sum**0.5))

In [ ]:
cosine_scores_3 = list(reversed(sorted(cosine_score_3.items(), key = lambda kv:(kv[1], kv[0]))))
for i in range(5):
  print(file_words[cosine_scores_3[i][0]])

fable.txt
hop-frog.poe
monkking.txt
6ablemen.txt
pussboot.txt


NORMALISATION

In [ ]:
cosine_score_4={}
for i in range(467):
  sum=0
  prod=0
  prod_sum=0
  for j in tfidf_norm:
    sum+=tfidf_norm[j][i]**2
    prod=prod+query_vector[j]*tfidf_norm[j][i]
  cosine_score_4[i]=(prod/(sum**0.5))

In [ ]:
cosine_scores_4 = list(reversed(sorted(cosine_score_4.items(), key = lambda kv:(kv[1], kv[0]))))
for i in range(5):
  print(file_words[cosine_scores_4[i][0]])

monkking.txt
pussboot.txt
mario.txt
6ablemen.txt
lpeargrl.txt


DOUBLE_NORMALISATION


In [ ]:
cosine_score_5={}
for i in range(467):
  sum=0
  prod=0
  prod_sum=0
  for j in tfidf_doublenorm:
    sum+=tfidf_doublenorm[j][i]**2
    prod=prod+query_vector[j]*tfidf_doublenorm[j][i]
  cosine_score_5[i]=(prod/(sum**0.5))

In [ ]:
cosine_scores_5 = list(reversed(sorted(cosine_score_5.items(), key = lambda kv:(kv[1], kv[0]))))
for i in range(5):
  print(file_words[cosine_scores_5[i][0]])

monkking.txt
mario.txt
pussboot.txt
lionwar.txt
19.lws
